In [1]:
import sys
import os
import numpy as np
import time
from imp import reload

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
    
from testing.inputs import *

new directory is: C:\Users\Luke\Desktop\Barracuda\AutomatedCE


## Step 1. Load the CE System with a Config File

Select one of the predesigned configurations (TE300, Eclipse Ti, Simulation). See an example config under "/config/test-system.cfg". 

Change the variable CONFIG_PATH to the absolute path where the config file resides. 

Be sure to make the string a raw string by prepending *'r'*. **Example:** r"C:\path\to\config\"*

In [2]:
CONFIG_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Eclipse.cfg"
from L3 import SystemsBuilder
reload(SystemsBuilder)

ce_system = SystemsBuilder.CESystem()
ce_system.load_config(CONFIG_PATH)
ce_system.open_controllers()
ce_system.startup_utilities()

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\MicroControlServer.py


 Change System Config setting if thorlabs is needed. 


{'outlet_pressure': <L2.PressureControl.ArduinoPressure object at 0x00000225C574C5C8>, 'xy_stage': <L2.XYControl.PycromanagerXY object at 0x00000225C574C508>, 'objective': <L2.ZControl.PycromanagerZ object at 0x00000225C574C688>, 'outlet_z': <L2.ZControl.ArduinoZ object at 0x00000225C574C708>, 'inlet_z': <L2.ZControl.ArduinoZ object at 0x00000225C574C608>, 'filter_wheel': None, 'excitation_shutter': None, 'camera': <L2.CameraControl.PycromanagerControl object at 0x00000225C574CB48>, 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply object at 0x00000225C574C808>, 'detector': <L2.DetectorControl.PhotomultiplierDetector object at 0x00000225C574C908>}


# Step 2. Prepare the Capillary for Electrophoresis

We need to prep the capillary for CE. This involves rinsing the capillary surface and removing contaminants as well as re-establsihing a deprotonated silanol layer. We also need to align the capillary with detection objective. 

### Step 2.1 Rinse the capillary (Repeatable)

Rinse the capillary according to your method. A good place to start is 10 min with HCl, 30 min with NaOH, 2 min with water. If this is your first rinse of the day,rinse with an empty vial first to make sure there are no leaks on the outlet side of the apparatus. 

In [13]:
# Start Rinse
ce_system.outlet_pressure.rinse_pressure()

In [14]:
# Stop Rinse 
ce_system.outlet_pressure.release()

### Step 2.2 Align the capillary with fluorescein. 

Place a vial with 125 nM fluorescein in the outlet. Once the pressure is rinsing adjust the capillary position using the two set screws as well as the adjustable objective collar until the signal is maximized for the detector being used. (PMT gain voltage should be set to 250 mV, or the lowest setting with appreciable gain). 

In [8]:
# Run the CE View
from L4 import DisplayWindows
reload(DisplayWindows)
%matplotlib qt
ce_disp = DisplayWindows.PLTCEDisplay(ce_system)
ce_disp.show()
ce_system.detector.start()
ce_disp.start_live_view()

In [ ]:
resp = get_yes_no("Is fluorescein in the outlet and ready to be rinsed with?")

if resp:
    ce_system.outlet_pressure.rinse_pressure()
    print("Adjust the capillary inlet while maximizing the signal")
if not resp:
    print("Please insert the fluorscein in the outlet, then re-run this cell.")

When finished. Record the maximum signal, along with the detector used, its gain setting (or bias when using SiPM), and the fluorescein concentration in the calibration log (located on the desktop). 

Run the next cell to stop the pressure rinse. 

In [ ]:
ce_system.outlet_pressure.release()

### Step 2.2 Post Fluorescein Rinse

Take a vial and add close to 4 mL of H20. Insert it into the outlet and then remove it. This removes any residual Fluorescein from the capillary, electrode, or other surfaces.  

Repeat with a second vial, with liquid slightly higher or the same height. 

Finally switch with separation buffer and rinse for 10 min. 

In [ ]:
resp = get_yes_no("Have you done the pre-fluorescein rinses?")
if not resp:
    print("Please follow instructions in step 2.2")
else:
    resp = get_yes_no("Is the separation buffer placed in the outlet?")
    if resp:
        ce_system.outlet_pressure.rinse_pressure()
        print("Run the next cell when finished")
    else:
        print("Please place the separation buffer in the outlet before rinsing.")

In [ ]:
ce_system.outlet_pressure.rinse_pressure()

# Step 3. Calibrate the Capillary Inlet

The method for calibration depends on the z-stage used to raise the capillary inlet. Select the correct route for the hardware employed. 

## Thorlabs Kinesis Z Stage

### Step 3.1 Lower the Capillary (thorlabs)
Move the capillary to a safe location where it will not be crushed if the stage lowers too far. Using the controls on the Z-stage itself, lower the capillary till as far as it will go. 


In [ ]:
resp = get_yes_no("Have you lowered the Kinesis Z stage to its lowest position? (y/n):  ")
if not resp:
    print("Please make sure the Kinesis stage is lowered to its lowest position before continuing")
else:
    print("Proceed to the next cell.")
    

### Step 3.2 Home the Capillary (thorlabs)

Thorlabs needs to register the home position. In the next cell we will call the go_home command to home the capillary. 

In [ ]:
ce_system.inlet_z.homing()

### Step 3.1 (Arduino/Optics-Focus)
The optics focus moves the capillary all the way up during the home step. Once this is done, move the capillary to a safe position and lower (with 1-5mm increments) until the capillary is at 0 mm. 

In [ ]:
ce_system.inlet_z.set_rel_z(-5)

In [ ]:
ce_system.inlet_z.set_z(0)

In [ ]:
resp = get_yes_no("Have you lowered the Arduino Z stage to 0 mm? (y/n):  ")
if not resp:
    print("Please make sure the Aduino stage is lowered to 1 mm position before continuing")
else:
    print("Proceed to the step 3.3 ")

### Step 3.3 Adjust Capillary Position (all stages)

Adjust the capillary position by sliding the detection apparatus up or down the optical support post. The capillary should be level with the bottom of the template for single-cell assays, or the top-of the template for well-based assays. 

*Aligning to the top of the template is a minor safety feature to prevent the user from sending the capillary to Zero position unexpetedly and crashing the capillary against the bottom of the template. However, capillary needs to be able to reach at least the glass slide of the cell reservoir for single-cell assays*


In [ ]:
resp = get_yes_no("Have you adjusted the capillary height manually so that it is flush (level) with the top or bottom of the template? (y/n):  ")
if not resp:
    print("Please make sure the capillary is moved to the correct height with respect to the template before continuing")
else:
    print("Proceed to the next cell")

### Step 3.4 Raise the capillary to a safe height

The next cell will raise the capillary height to a location where it ***should*** be safe for any XY movements. Verify the capillary will not crash at this new height. 

In [ ]:
target_z = get_float("Please enter a safe height to move the stage to: ")
# Move the capillary up and wait for it to stop
ce_system.inlet_z.set_z(target_z)
ce_system.inlet_z.wait_for_target(target_z)

resp = get_yes_no("Is the capillary at a safe location?: ")
if resp:
    print("Continue to next step")
else:
    print("Please raise the capillary to an appropriate height")

# Step 4. Home the XY Stage

This next step provides the calibration for the XY axis for the XY stage and capillary inlet. 


### Step 4.1 (Nikon Eclipse)

The nikon eclipse has a home position defined to the center of the template. It is best to use this as the home position. For the capillary alignment, align the capillary concentrically with the objective. The capillary should be within 0.5 mm of the center of the objective if using PCR tubes at any point. 

In [ ]:
resp = get_yes_no("Is the capillary aligned with the objective?")
if resp:
    print("Continue to next step")
else:
    print("Please raise align the capillary with the objective")

# Step 5 Verify your calibration

Steps 3-4 calibrate the capillaries position with the microscope's position. Now we need to verify that our alignment is correct. To do this we will move the capillary to a well and make sure that it is centered over the well. Then we will lower the capillary to a height specified by the method to make sure the capillary can reach the liquid. 


### Step 5.1. Load the Template File

The template file tells the program where to move the XY stage for each well as well as how high to raise the capillary inlet during a move. Change the variable TEMPLATE_PATH to the absolute path where the template.txt file resides. 

In [3]:
TEMPLATE_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\EclipseCellTemplate.txt"
from L4 import AutomatedControl
reload(AutomatedControl)

auto_program = AutomatedControl.AutoRun(ce_system)
auto_program.set_template(TEMPLATE_PATH)

### Step 5.2 Raise the capillary to a safe height 

In [10]:
z = get_float("Set the capillary to a safe travel height: ")
ce_system.inlet_z.set_z(z)
print("Proceed to next cell")

Set the capillary to a safe travel height:  24
Proceed to next cell


### Step 5.2 Center the Capillary over a well

In [ ]:
name = get_value("Please enter the well name to move to (press enter to see a list): ", auto_program.template.wells.keys())
well = auto_program.template.wells[name]
ce_system.xy_stage.set_xy(well.xy)
resp = input("Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.")


### Step 5.3 Lower the capillary to the height described in the method file


In [ ]:
height_z = get_float("What height should the capillary be lowered to?")
prev_height = ce_system.inlet_z.read_z()
ce_system.inlet_z.set_z(height_z)
resp = get_yes_no("Is the capillary touching the liquid in this well? ")
if not resp:
    print("Adjust the height of the capillary manually (and carefully!) so that the capillary is touching the liquid")
else:
    print("Calibration Finished")
ce_system.inlet_z.set_z(prev_height)

# Step 6. Load a method

Load a method file. Verify your method file in excel and make sure the run is set up as intended. You can add multiple files to the same auto_program


In [4]:
file_to_add = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\EclipseMethodTest.txt"
auto_program.add_method(file_to_add)


If you need to reset the methods (remove and start over) run the next cell

In [ ]:
auto_program.methods=[]

### Step 6.2  Run your method (repeatable)

When you are ready to start your method, run the next cell. 

In [9]:
auto_program.start_run()

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\DisplayWindows.py:270: UserWarning: Attempting to set identical left == right == 0.0 results in singular transformations; automatically expanding.
  ax1.set_xlim(min(data['time_data']), max(data['time_data']))


# Step 7. End of Day

1. Repeat Step 2.1, this time rinse for 5 min with water, then 30 min with air. Run the next cell when you are finished rinsing.
2. Turn off all equipment. 

In [15]:
ce_system.close_controllers()

In [ ]:
ce_system.open_controllers()

In [ ]:
cam_disp = DisplayWindows.PLTMicroscopeDisplay(ce_system)

In [ ]:
cam_disp.show()

In [ ]:
cam_disp.stop_live_image()

In [ ]:
cam_disp.percentiles = 0

In [ ]:
ce_system.camera.set_exposure(45)

In [ ]:
cam_disp.live_image()